# Playground and experiments

_This is not the notebook you want to read..._

In [4]:
import xarray
from pathlib import Path
from IPython.display import display

data_dir = Path("data")


## What happens if we combine two DataArrays with different coordinates in a Dataset?

In [14]:
da1 = xarray.open_rasterio(data_dir / "grassland_25.tiff")
da2 = xarray.open_rasterio(data_dir / "small_woody_features_27.tiff")
da3 = xarray.open_rasterio(data_dir / "tree_cover_density_25.tiff")

In [7]:
print(da1)

<xarray.DataArray (band: 1, y: 1245, x: 1612)>
[2006940 values with dtype=uint8]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 6.148e+06 6.148e+06 6.148e+06 ... 6.117e+06 6.117e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
Attributes:
    transform:      (25.00772334211627, 0.0, 1264675.9106559341, 0.0, -24.995...
    crs:            +init=epsg:3857
    res:            (25.00772334211627, 24.995318193712)
    is_tiled:       1
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    DataType:       Generic


In [8]:
print(da2)

<xarray.DataArray (band: 1, y: 1153, x: 1493)>
[1721429 values with dtype=uint8]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 6.148e+06 6.148e+06 6.148e+06 ... 6.117e+06 6.117e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
Attributes:
    transform:      (27.000971217341885, 0.0, 1264675.9106559341, 0.0, -26.98...
    crs:            +init=epsg:3857
    res:            (27.000971217341885, 26.989740807607493)
    is_tiled:       1
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    DataType:       Generic


In [17]:
ds = xarray.Dataset({"grass": da1, "swf": da2})

In [18]:
print(ds)

<xarray.Dataset>
Dimensions:  (band: 1, x: 3105, y: 2397)
Coordinates:
  * y        (y) float64 6.117e+06 6.117e+06 6.117e+06 ... 6.148e+06 6.148e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
  * band     (band) int64 1
Data variables:
    grass    (band, y, x) float64 0.0 nan 0.0 nan 0.0 ... 0.0 nan 0.0 nan 0.0
    swf      (band, y, x) float64 nan nan nan nan nan ... nan nan nan nan nan


-> we get the union of coordinates in the Dataset

## What about identical coordinates?

In [28]:
ds = xarray.Dataset({"grass": da1, "trees": da3})

In [30]:
print(ds)

assert (ds.coords["x"].values == da1.coords["x"].values).all()
assert (ds.coords["y"].values == da1.coords["y"].values).all()
assert (ds.coords["x"].values == da3.coords["x"].values).all()
assert (ds.coords["y"].values == da3.coords["y"].values).all()

<xarray.Dataset>
Dimensions:  (band: 1, x: 1612, y: 1245)
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 6.148e+06 6.148e+06 6.148e+06 ... 6.117e+06 6.117e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
Data variables:
    grass    (band, y, x) uint8 ...
    trees    (band, y, x) uint8 ...


-> we get one Dataset with the same coordinates as the input DataArrays

## Interpolate to join arrays with different coordinates

This fails: Tries to interpolate on 'band', which has only one value.

In [37]:
da2_resampled = da2.interp_like(da1)

ValueError: x and y arrays must have at least 2 entries

Squeeze out 'band' dimension before resampling:

In [42]:
da2_resampled = da2.squeeze('band').interp_like(da1, method='nearest')


In [44]:
print(da2_resampled)

assert (da2_resampled.coords["x"].values == da1.coords["x"].values).all()
assert (da2_resampled.coords["y"].values == da1.coords["y"].values).all()

<xarray.DataArray (y: 1245, x: 1612)>
array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan, 255., 255., ..., 255., 255.,  nan],
       [ nan, 255., 255., ..., 255., 255.,  nan],
       ...,
       [ nan, 255., 255., ..., 255., 255.,  nan],
       [ nan, 255., 255., ..., 255., 255.,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]])
Coordinates:
    band     int64 1
  * y        (y) float64 6.148e+06 6.148e+06 6.148e+06 ... 6.117e+06 6.117e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
Attributes:
    transform:      (27.000971217341885, 0.0, 1264675.9106559341, 0.0, -26.98...
    crs:            +init=epsg:3857
    res:            (27.000971217341885, 26.989740807607493)
    is_tiled:       1
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    DataType:       Generic


Now we can nicely merge all three DataArrays into one Dataset

In [46]:
ds = xarray.Dataset({"grass": da1, "swf": da2_resampled, "trees": da3})

print(ds)

assert (ds.coords["x"].values == da1.coords["x"].values).all()
assert (ds.coords["y"].values == da1.coords["y"].values).all()

<xarray.Dataset>
Dimensions:  (band: 1, x: 1612, y: 1245)
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 6.148e+06 6.148e+06 6.148e+06 ... 6.117e+06 6.117e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
Data variables:
    grass    (band, y, x) uint8 ...
    swf      (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan nan
    trees    (band, y, x) uint8 ...


In [47]:
ds.max()

<xarray.Dataset>
Dimensions:  ()
Data variables:
    grass    uint8 1
    swf      float64 255.0
    trees    uint8 100

In [50]:
ds["grass"]

<xarray.DataArray 'grass' (band: 1, y: 1245, x: 1612)>
array([[[0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0],
        ...,
        [0, 0, ..., 0, 0],
        [0, 0, ..., 0, 0]]], dtype=uint8)
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 6.148e+06 6.148e+06 6.148e+06 ... 6.117e+06 6.117e+06
  * x        (x) float64 1.265e+06 1.265e+06 1.265e+06 ... 1.305e+06 1.305e+06
Attributes:
    transform:      (25.00772334211627, 0.0, 1264675.9106559341, 0.0, -24.995...
    crs:            +init=epsg:3857
    res:            (25.00772334211627, 24.995318193712)
    is_tiled:       1
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area
    DataType:       Generic